# Med3d


In [83]:
from functools import partial
from typing import Literal, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

__all__ = [
    "ResNet",
    "resnet10",
    "resnet18",
    "resnet34",
    "resnet50",
    "resnet101",
    "resnet152",
    "resnet200",
    "get_resnet",
]


def conv3x3x3(in_planes: int, out_planes: int, stride: int = 1, dilation: int = 1) -> nn.Conv3d:
    """3x3x3 convolution with padding"""
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        dilation=dilation,
        stride=stride,
        padding=dilation,
        bias=False,
    )


def downsample_basic_block(
    x: torch.Tensor, planes: int, stride: int, no_cuda: bool = False
) -> torch.Tensor:
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1), out.size(2), out.size(3), out.size(4)
    ).zero_()
    if not no_cuda:
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        dilation: int = 1,
        downsample: bool = None,
    ):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride=stride, dilation=dilation)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes, dilation=dilation)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        dilation: int = 1,
        downsample: bool = None,
    ):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(
            planes,
            planes,
            kernel_size=3,
            stride=stride,
            dilation=dilation,
            padding=dilation,
            bias=False,
        )
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(
        self,
        block: Union[Bottleneck, BasicBlock],
        layers: list[int],
        num_classes: int = 10,
        shortcut_type: Literal["A", "B"] = "B",
        no_cuda: bool = False,
    ):
        self.inplanes = 64
        self.no_cuda = no_cuda
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(
            1, 64, kernel_size=7, stride=(2, 2, 2), padding=(3, 3, 3), bias=False
        )

        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], shortcut_type, stride=1, dilation=2)
        self.layer4 = self._make_layer(block, 512, layers[3], shortcut_type, stride=1, dilation=4)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        # fc_in_features = 2048 if block == Bottleneck else 512
        self.fc = nn.Linear(self.inplanes, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal(m.weight, mode="fan_out")
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(
        self,
        block: Union[Bottleneck, BasicBlock],
        planes: int,
        blocks: int,
        shortcut_type: Literal["A", "B"],
        stride: int = 1,
        dilation: int = 1,
    ):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == "A":
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride,
                    no_cuda=self.no_cuda,
                )
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False,
                    ),
                    nn.BatchNorm3d(planes * block.expansion),
                )

        layers = []
        layers.append(
            block(self.inplanes, planes, stride=stride, dilation=dilation, downsample=downsample)
        )
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation))

        return nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet10(**kwargs):
    """Constructs a ResNet-18 model."""
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model."""
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model."""
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model."""
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model."""
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model."""
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model."""
    model = ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model


def get_resnet(model_depth: int = 18, **kwargs):
    """
    Get a ResNet model based on the specified depth.

    Args:
        model_depth (int): Depth of the ResNet model (e.g., 10, 18, 34, 50, 101, 152, 200).
        **kwargs: Additional arguments for the ResNet constructor.

    Returns:
        nn.Module: The ResNet model.
    """
    if model_depth == 10:
        return resnet10(**kwargs)
    elif model_depth == 18:
        return resnet18(**kwargs)
    elif model_depth == 34:
        return resnet34(**kwargs)
    elif model_depth == 50:
        return resnet50(**kwargs)
    elif model_depth == 101:
        return resnet101(**kwargs)
    elif model_depth == 152:
        return resnet152(**kwargs)
    elif model_depth == 200:
        return resnet200(**kwargs)
    else:
        raise ValueError(f"Unsupported ResNet depth: {model_depth}")


In [84]:
from typing import Literal

import torch
from huggingface_hub import hf_hub_download

from src.modeling.base_model import VertebraeClassifier


class Med3DClassifier(VertebraeClassifier):
    def __init__(
        self,
        num_classes: int,
        model_depth: int = 18,
        shortcut_type: Literal["A", "B"] = "B",
        load_pretrained: bool = True,
        freeze_backbone: bool = True,
        device: torch.device = torch.device("cuda"),
    ) -> None:
        """
        Classifier using Med3D ResNet backbone with optional pretrained weights.

        Args:
            num_classes (int): Number of output classes.
            model_depth (int): Depth of ResNet (e.g., 10, 18, 34...).
            shortcut_type (Literal['A', 'B']): Type of shortcut connection.
            load_pretrained (bool): Whether to load pretrained weights from Med3D.
            freeze_backbone (bool): Whether to freeze the backbone weights.
        """
        super().__init__(num_classes)
        self.device = device

        self.model = get_resnet(
            model_depth=model_depth,
            num_classes=num_classes,
            shortcut_type=shortcut_type,
        )

        if load_pretrained is not None:
            self._load_med3d_weights(model_depth)

        if freeze_backbone:
            for param in self.model.parameters():
                param.requires_grad = False
            for param in self.model.fc.parameters():
                param.requires_grad = True

    def _load_med3d_weights(self, model_depth: int) -> None:
        """
        Load pretrained Med3D weights from HuggingFace, skipping the classification layer.
        """
        hf_mapping = {
            10: ("TencentMedicalNet/MedicalNet-ResNet10", "resnet_10.pth"),
            18: ("TencentMedicalNet/MedicalNet-ResNet18", "resnet_18.pth"),
            34: ("TencentMedicalNet/MedicalNet-ResNet34", "resnet_34.pth"),
            50: ("TencentMedicalNet/MedicalNet-ResNet50", "resnet_50.pth"),
            101: ("TencentMedicalNet/MedicalNet-ResNet101", "resnet_101.pth"),
            152: ("TencentMedicalNet/MedicalNet-ResNet152", "resnet_152.pth"),
            200: ("TencentMedicalNet/MedicalNet-ResNet200", "resnet_200.pth"),
        }

        if model_depth not in hf_mapping:
            raise ValueError(f"No pretrained weights available for model depth {model_depth}")

        repo_id, filename = hf_mapping[model_depth]
        weight_path = hf_hub_download(repo_id=repo_id, filename=filename, cache_dir="../models/med3d")

        state_dict = torch.load(weight_path, map_location=self.device)["state_dict"]
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items() if not k.startswith("conv-seg")}            

        missing, unexpected = self.model.load_state_dict(state_dict, strict=False)
        print(
            f"[Med3D] Loaded weights with {len(missing)} missing and {len(unexpected)} unexpected keys. {len(state_dict)} total keys loaded." 
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)


In [ ]:
med3D = Med3DClassifier(10, 34)
med3D.to("cpu")
summary(med3D, input_size=(1, 64, 64, 64), device="cpu")

`nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.


resnet_34.pth:   0%|          | 0.00/253M [00:00<?, ?B/s]

In [86]:
## save model 
model_path = "med3d_resnet10.pth"
torch.save(model.state_dict(), model_path)


In [87]:
state_dict_2 = torch.load(model_path, map_location="cpu")

In [88]:
for k, v in state_dict_2.items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: {v.shape}")
    else:
        print(f"{k}: {type(v)}")  # Print type for non-tensor values

convInit.conv.weight: torch.Size([32, 1, 3, 3, 3])
down_layers.0.1.norm1.weight: torch.Size([32])
down_layers.0.1.norm1.bias: torch.Size([32])
down_layers.0.1.norm2.weight: torch.Size([32])
down_layers.0.1.norm2.bias: torch.Size([32])
down_layers.0.1.conv1.conv.weight: torch.Size([32, 32, 3, 3, 3])
down_layers.0.1.conv2.conv.weight: torch.Size([32, 32, 3, 3, 3])
down_layers.1.0.conv.weight: torch.Size([64, 32, 3, 3, 3])
down_layers.1.1.norm1.weight: torch.Size([64])
down_layers.1.1.norm1.bias: torch.Size([64])
down_layers.1.1.norm2.weight: torch.Size([64])
down_layers.1.1.norm2.bias: torch.Size([64])
down_layers.1.1.conv1.conv.weight: torch.Size([64, 64, 3, 3, 3])
down_layers.1.1.conv2.conv.weight: torch.Size([64, 64, 3, 3, 3])
down_layers.1.2.norm1.weight: torch.Size([64])
down_layers.1.2.norm1.bias: torch.Size([64])
down_layers.1.2.norm2.weight: torch.Size([64])
down_layers.1.2.norm2.bias: torch.Size([64])
down_layers.1.2.conv1.conv.weight: torch.Size([64, 64, 3, 3, 3])
down_layers.1

# Monai

In [59]:
import os
import torch
from monai.bundle import ConfigParser

model_dir = "../models/monai_seg"
model_file = "models/model.pt"  # zakładamy, że w folderze models jest plik model.pt

config_path = os.path.join(model_dir, "configs", "inference.json")
parser = ConfigParser()
parser.read_config(config_path)

model = parser.get_parsed_content("network_def")
weights = torch.load(os.path.join(model_dir, model_file), map_location="cpu")
if isinstance(weights, dict) and "state_dict" in weights:
    weights = weights["state_dict"]
model.load_state_dict(weights)
model.eval()


SegResNet(
  (act_mod): ReLU(inplace=True)
  (convInit): Convolution(
    (conv): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
  )
  (down_layers): ModuleList(
    (0): Sequential(
      (0): Identity()
      (1): ResBlock(
        (norm1): GroupNorm(8, 32, eps=1e-05, affine=True)
        (norm2): GroupNorm(8, 32, eps=1e-05, affine=True)
        (act): ReLU(inplace=True)
        (conv1): Convolution(
          (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
        (conv2): Convolution(
          (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
      )
    )
    (1): Sequential(
      (0): Convolution(
        (conv): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      )
      (1): ResBlock(
        (norm1): GroupNorm(8, 64, eps=1e-05, affine=True)
        (norm2): GroupNorm(8, 64, eps=1e-05, a

In [60]:
from torch import nn

model.pool = nn.AdaptiveAvgPool3d((1, 1, 1))
model.flatten = nn.Flatten()
model.classifier = nn.Linear(256, 10)

def forward(model, x: torch.Tensor) -> torch.Tensor:
    x = model.convInit(x)
    for down in model.down_layers:
        x = down(x)
    x = model.pool(x)
    x = model.flatten(x)
    x = model.classifier(x)
    return x

In [61]:
from torchsummary import summary


model.to("cpu")
summary(model, input_size=(1, 64, 64, 64), device="cpu")



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 32, 64, 64, 64]             864
         Dropout3d-2       [-1, 32, 64, 64, 64]               0
          Identity-3       [-1, 32, 64, 64, 64]               0
         GroupNorm-4       [-1, 32, 64, 64, 64]              64
              ReLU-5       [-1, 32, 64, 64, 64]               0
            Conv3d-6       [-1, 32, 64, 64, 64]          27,648
         GroupNorm-7       [-1, 32, 64, 64, 64]              64
              ReLU-8       [-1, 32, 64, 64, 64]               0
            Conv3d-9       [-1, 32, 64, 64, 64]          27,648
         ResBlock-10       [-1, 32, 64, 64, 64]               0
           Conv3d-11       [-1, 64, 32, 32, 32]          55,296
        GroupNorm-12       [-1, 64, 32, 32, 32]             128
             ReLU-13       [-1, 64, 32, 32, 32]               0
           Conv3d-14       [-1, 64, 32,

In [62]:
import torch
from monai.bundle import ConfigParser
from torch import nn
from src.modeling.base_model import VertebraeClassifier
from src.config import SEG_MODEL_DIR


class SegResNetClassifier(VertebraeClassifier):
    def __init__(self, num_classes: int = 10) -> None:
        """
        Classifier using MONAI's SegResNet backbone.

        Args:
            num_classes (int): Number of output classes.
        """
        super().__init__(num_classes)
        self._load_monai_model()
        self._delete_decoder_layers()
        self._add_classifier_layers()

    def _load_monai_model(self) -> None:
        """
        Load the MONAI SegResNet model from the specified directory.
        """
        config_path = SEG_MODEL_DIR / "configs/inference.json"
        parser = ConfigParser()
        parser.read_config(config_path)

        self.model = parser.get_parsed_content("network_def")
        weights = torch.load(SEG_MODEL_DIR / "models/model.pt", map_location="cpu")
        if isinstance(weights, dict) and "state_dict" in weights:
            weights = weights["state_dict"]
        self.model.load_state_dict(weights)


    def _delete_decoder_layers(self) -> None:
        """
        Remove decoder layers from the model to keep only the encoder part.
        """
        for attr in ["up_samples", "up_conv", "final_conv"]:
            if hasattr(self.model, attr):
                delattr(self.model, attr)
    
    def _add_classifier_layers(self) -> None:
        """
        Add classifier layers to the model.
        """
        self.model.pool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.model.flatten = nn.Flatten()
        self.model.classifier = nn.Linear(256, self.num_classes)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        x = self.model.convInit(x)
        for down in self.model.down_layers:
            x = down(x)
        x = self.model.pool(x)
        x = self.model.flatten(x)
        x = self.model.classifier(x)
        return x            

In [63]:
model2 = SegResNetClassifier(num_classes=10)
model2.to("cpu")
summary(model2, input_size=(1, 64, 64, 64), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 32, 64, 64, 64]             864
          Identity-2       [-1, 32, 64, 64, 64]               0
         GroupNorm-3       [-1, 32, 64, 64, 64]              64
              ReLU-4       [-1, 32, 64, 64, 64]               0
            Conv3d-5       [-1, 32, 64, 64, 64]          27,648
         GroupNorm-6       [-1, 32, 64, 64, 64]              64
              ReLU-7       [-1, 32, 64, 64, 64]               0
            Conv3d-8       [-1, 32, 64, 64, 64]          27,648
          ResBlock-9       [-1, 32, 64, 64, 64]               0
           Conv3d-10       [-1, 64, 32, 32, 32]          55,296
        GroupNorm-11       [-1, 64, 32, 32, 32]             128
             ReLU-12       [-1, 64, 32, 32, 32]               0
           Conv3d-13       [-1, 64, 32, 32, 32]         110,592
        GroupNorm-14       [-1, 64, 32,

In [82]:
med3D

Med3DClassifier(
  (model): ResNet(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)


In [4]:
from src.modeling.model_factory import create_model

def params_to_mb(n_params: int) -> float:
    return (n_params * 4) / (1024 ** 2)

sizes = [10, 18, 34, 50, 101, 152, 200]

for size in sizes:
    med3d_frezed = create_model(
        model_type='med3d',
        num_classes=10,
        model_depth=size,
        device="cpu",
        freeze_backbone=True
    )

    trainable = sum(p.numel() for p in med3d_frezed.parameters() if p.requires_grad)
    total = sum(p.numel() for p in med3d_frezed.parameters())

    print(f"Size {size}: Trainable = {params_to_mb(trainable):.2f} MB, Total = {params_to_mb(total):.2f} MB")


`nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.


[Med3D] Loaded weights with 62 missing and 1 unexpected keys.
Size 10: Trainable = 0.02 MB, Total = 54.78 MB
[Med3D] Loaded weights with 102 missing and 1 unexpected keys.
Size 18: Trainable = 0.02 MB, Total = 126.51 MB
[Med3D] Loaded weights with 182 missing and 1 unexpected keys.
Size 34: Trainable = 0.02 MB, Total = 242.14 MB
[Med3D] Loaded weights with 267 missing and 1 unexpected keys.
Size 50: Trainable = 0.08 MB, Total = 176.15 MB
[Med3D] Loaded weights with 522 missing and 1 unexpected keys.
Size 101: Trainable = 0.08 MB, Total = 325.10 MB
[Med3D] Loaded weights with 777 missing and 1 unexpected keys.
Size 152: Trainable = 0.08 MB, Total = 447.77 MB
[Med3D] Loaded weights with 1017 missing and 1 unexpected keys.
Size 200: Trainable = 0.08 MB, Total = 482.86 MB
